In [ ]:
import cupy as cp
import numpy as np
import h5py
from decorrelation.shp import ks_test
from decorrelation.co import emperical_co

In [ ]:
rslc = cp.load('./rslc.npy')

In [ ]:
h5f = h5py.File('decorrelation.h5', 'w')

## rslc

In [ ]:
h5f.create_dataset('rslc', data=cp.asnumpy(rslc))

<HDF5 dataset "rslc": shape (2500, 1834, 17), type "<c8">

coherence

In [ ]:
az_half_win = 5
r_half_win = 5
az_win = 2*az_half_win+1
r_win = 2*r_half_win+1

In [ ]:
rmli = cp.abs(rslc)**2
sorted_rmli = cp.sort(rmli,axis=-1)
dist, p = ks_test(sorted_rmli,az_half_win=az_half_win,r_half_win=r_half_win)

In [ ]:
is_shp = (p < 0.05) & (p >= 0.0)

In [ ]:
cov,coh = emperical_co(rslc,is_shp)

In [ ]:
h5f.create_dataset('coherence', data=cp.asnumpy(coh))

<HDF5 dataset "coherence": shape (2500, 1834, 17, 17), type "<c8">

In [ ]:
h5f.close()